In [25]:
# Cell 1: Import necessary libraries
import tensorflow as tf
from keras.layers import Layer
from keras.models import load_model
import os
import glob

In [33]:
inputs = tf.keras.Input(shape=(150, 150, 3))
# Cell 2: Define the AdaptiveConv2D layer
class AdaptiveConvLayer(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size):
        super(AdaptiveConvLayer, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size

    def build(self, input_shape):
        self.conv_weights = self.add_weight(shape=(self.kernel_size, self.kernel_size, input_shape[-1], self.filters),
                                            initializer='random_normal',
                                            trainable=True)
        super(AdaptiveConvLayer, self).build(input_shape)

    def call(self, inputs):
        output = tf.nn.conv2d(inputs, self.conv_weights, strides=[1, 1, 1, 1], padding='SAME')
        return output

    # Generate Kernel Weights for Iteration and Return to Model
    def get_kernel_weights(self):
        return self.conv_weights


In [34]:

# Cell 3: Define the CustomModel class
class CustomModel(tf.keras.Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.adaptive_conv = AdaptiveConvLayer(filters=3, kernel_size=3)
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(6, activation='softmax')

    def call(self, inputs):
        x = self.adaptive_conv(inputs)
        x = self.flatten(x)
        x = self.dense(x)
        return x

    def get_kernel_weights(self):
        return self.adaptive_conv.get_kernel_weights()




In [35]:
# Define a custom callback to save kernel weights after each epoch
class SaveKernelWeightsCallback(tf.keras.callbacks.Callback):
    def __init__(self, model, csv_writer):
        super(SaveKernelWeightsCallback, self).__init__()
        self.model = model
        self.csv_writer = csv_writer

    def on_epoch_end(self, epoch, logs=None):
        # Get the kernel weights of the AdaptiveConvLayer
        kernel_weights = self.model.get_kernel_weights()

        # Flatten the kernel weights for writing to CSV
        flattened_weights = kernel_weights.numpy().flatten()

        # Prepare the row to write to CSV
        row = [epoch] + list(flattened_weights)

        # Write the row to the CSV file
        self.csv_writer.writerow(row)

In [36]:
# Create an instance of the custom model
model = CustomModel()

In [38]:
# Initialize the CSV file and writer
csv_file = open('weights.csv', mode='w', newline='')
csv_writer = csv.writer(csv_file)

# Write the header row in the CSV file
header = ['Epoch'] + [f'Weight_{i+1}' for i in range(model.adaptive_conv.kernel_size**2)]
csv_writer.writerow(header)

# Create the callback to save kernel weights after each epoch
save_weights_callback = SaveKernelWeightsCallback(model, csv_writer)

train_data_dir = "seg_train"
batch_size = 32
image_size = (150, 150)

# Create the train_dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    batch_size=batch_size,
    image_size=image_size,
    validation_split=0.2,
    subset="training",
    seed=123
)

# Compile
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

# Train the model with the callback
model.fit(train_dataset, epochs=10, callbacks=[save_weights_callback],batch_size=32)

# Close the CSV file
csv_file.close()

Found 14034 files belonging to 6 classes.
Using 11228 files for training.
Epoch 1/10
351/351 [==============================] - 24s 69ms/step - loss: 92.1446 - accuracy: 0.3760
Epoch 2/10
351/351 [==============================] - 18s 50ms/step - loss: 12.3016 - accuracy: 0.6413
Epoch 3/10
351/351 [==============================] - 21s 60ms/step - loss: 5.5492 - accuracy: 0.7789
Epoch 4/10
351/351 [==============================] - 19s 53ms/step - loss: 3.5203 - accuracy: 0.8408
Epoch 5/10
351/351 [==============================] - 17s 49ms/step - loss: 2.5015 - accuracy: 0.8758
Epoch 6/10
351/351 [==============================] - 18s 51ms/step - loss: 2.5008 - accuracy: 0.8803
Epoch 7/10
351/351 [==============================] - 18s 51ms/step - loss: 2.5284 - accuracy: 0.8915
Epoch 8/10
351/351 [==============================] - 19s 55ms/step - loss: 2.4209 - accuracy: 0.8959
Epoch 9/10
351/351 [==============================] - 16s 46ms/step - loss: 2.3035 - accuracy: 0.9039
Epoch 

In [32]:
# Compile
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [6]:
#Train
model.fit(train_dataset, epochs=10, batch_size=32)

Epoch 1/10
152/351 [===========>..................] - ETA: 19s - loss: 156.0372 - accuracy: 0.3454

KeyboardInterrupt: 

In [7]:
model.summary()

Model: "custom_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 adaptive_conv_layer (Adapti  multiple                 81        
 veConvLayer)                                                    
                                                                 
 flatten (Flatten)           multiple                  0         
                                                                 
 dense (Dense)               multiple                  405006    
                                                                 
Total params: 405,087
Trainable params: 405,087
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Save the model as a TensorFlow SavedModel
#model.save('TF_Models/')
model.save_weights('TF_Models/ECE50024_ACDA_TF_V3.h5')

Below is some extra stuff.

In [4]:

# Cell 4: Load the saved model
saved_model_path = 'ECE50024_ImgRec_TF_Model_V2.h5'
model = load_model(saved_model_path, custom_objects={'AdaptiveConv': AdaptiveConv})

In [5]:

# Cell 5: Define the path to the test image directory
test_image_dir = 'Intel_seg_pred'

In [6]:

# Cell 6: Load and preprocess the test images
img_paths = glob.glob(os.path.join(test_image_dir, '*.jpg'))
images = []
for img_path in img_paths:
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(150, 150))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.keras.applications.vgg16.preprocess_input(img_array)
    images.append(img_array)
images = tf.stack(images)

In [9]:

# Cell 7: Make predictions on the test images
predictions = model.predict(images)

25/25 [==============================] - 2s 71ms/step


In [10]:

# Cell 8: Display the predictions
for i, img_path in enumerate(img_paths):
    print('Image:', img_path)
    print('Predictions:', predictions[i])


Image: Intel_seg_pred\10060.jpg
Predictions: [0.         0.         0.         0.         0.         0.99999994]
Image: Intel_seg_pred\10082.jpg
Predictions: [0.         0.         0.         0.         0.         0.99999994]
Image: Intel_seg_pred\10090.jpg
Predictions: [0.         0.         0.         0.         0.         0.99999994]
Image: Intel_seg_pred\101.jpg
Predictions: [0.         0.         0.         0.         0.         0.99999994]
Image: Intel_seg_pred\10106.jpg
Predictions: [0.         0.         0.         0.         0.         0.99999994]
Image: Intel_seg_pred\10158.jpg
Predictions: [0.         0.         0.         0.         0.         0.99999994]
Image: Intel_seg_pred\10166.jpg
Predictions: [0.         0.         0.         0.         0.99999994 0.        ]
Image: Intel_seg_pred\10173.jpg
Predictions: [0.         0.         0.         0.         0.         0.99999994]
Image: Intel_seg_pred\10225.jpg
Predictions: [0.         0.         0.         0.         0.      

In [11]:
# Cell 8: Display the predictions
class_names = ['buildings', 'forest', 'street','sea','mountain','glacier']  # Replace with your own class names

for i, img_path in enumerate(img_paths):
    print('Image:', img_path)
    predictions_i = predictions[i]
    predicted_class_index = tf.argmax(predictions_i)
    predicted_class_name = class_names[predicted_class_index]
    print('Predicted class:', predicted_class_name)
    print('Predicted probabilities:', predictions_i)
    print()


Image: Intel_seg_pred\10060.jpg
Predicted class: glacier
Predicted probabilities: [0.         0.         0.         0.         0.         0.99999994]

Image: Intel_seg_pred\10082.jpg
Predicted class: glacier
Predicted probabilities: [0.         0.         0.         0.         0.         0.99999994]

Image: Intel_seg_pred\10090.jpg
Predicted class: glacier
Predicted probabilities: [0.         0.         0.         0.         0.         0.99999994]

Image: Intel_seg_pred\101.jpg
Predicted class: glacier
Predicted probabilities: [0.         0.         0.         0.         0.         0.99999994]

Image: Intel_seg_pred\10106.jpg
Predicted class: glacier
Predicted probabilities: [0.         0.         0.         0.         0.         0.99999994]

Image: Intel_seg_pred\10158.jpg
Predicted class: glacier
Predicted probabilities: [0.         0.         0.         0.         0.         0.99999994]

Image: Intel_seg_pred\10166.jpg
Predicted class: mountain
Predicted probabilities: [0.         0

In [12]:
import csv

# Define the output CSV file path
output_csv_file = 'ACDA_predictions.csv'

# Create a list to store the predictions and file names
output_data = []

# Iterate over the images and predictions
for i, img_path in enumerate(img_paths):
    predictions_i = predictions[i]
    predicted_class_index = tf.argmax(predictions_i)
    predicted_class_name = class_mapping[predicted_class_index]

    # Create a row for the current image
    row = [img_path, predicted_class_name] + predictions_i.tolist()

    # Append the row to the output data list
    output_data.append(row)

# Write the output data to the CSV file
with open(output_csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Image', 'Predicted Class'] + class_mapping.values())
    writer.writerows(output_data)

print('Predictions saved to', output_csv_file)


NameError: name 'class_mapping' is not defined